In [41]:
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup

In [31]:
league_id = "1125189236732284928"
url = f"https://api.sleeper.app/v1/league/{league_id}/drafts"
response = requests.get(url)
data = response.json()

draft_id = data[0]['draft_id'] #the startup draft
picks_url = f"https://api.sleeper.app/v1/draft/{draft_id}/picks"
picks = requests.get(picks_url).json()

entry_draft = pd.DataFrame(picks)
metadata_entry_draft = entry_draft['metadata'].apply(pd.Series)
entry_draft = pd.concat([entry_draft.drop(columns=['metadata']), metadata_entry_draft], axis=1)
entry_draft = entry_draft[['round', 'pick_no', 'roster_id', 'first_name', 'last_name', 'position', 'team']]
entry_draft = entry_draft.sort_values(by='pick_no')

rosters = requests.get(f"https://api.sleeper.app/v1/league/{league_id}/rosters").json()
users = requests.get(f"https://api.sleeper.app/v1/league/{league_id}/users").json()

roster_map = {r['roster_id']: r['owner_id'] for r in rosters}
user_map = {u['user_id']: u['display_name'] for u in users}

entry_draft['owner_id'] = entry_draft['roster_id'].map(roster_map)
entry_draft['owner_name'] = entry_draft['owner_id'].map(user_map)
del entry_draft['owner_id']
del entry_draft['roster_id']

entry_draft.to_csv('entry_draft.csv')

In [32]:
league_id = "1184039966157762560"
url = f"https://api.sleeper.app/v1/league/{league_id}/drafts"
response = requests.get(url)
data = response.json()
draft_id = data[0]['draft_id'] #the 2025 rookie draft
picks_url = f"https://api.sleeper.app/v1/draft/{draft_id}/picks"
picks = requests.get(picks_url).json()
rookie_draft = pd.DataFrame(picks)
metadata_rookie_draft = rookie_draft['metadata'].apply(pd.Series)
rookie_draft = pd.concat([rookie_draft.drop(columns=['metadata']), metadata_rookie_draft], axis=1)
rookie_draft = rookie_draft[['round', 'pick_no', 'roster_id', 'first_name', 'last_name', 'position', 'team']]
rookie_draft = rookie_draft.sort_values(by='pick_no')
rosters = requests.get(f"https://api.sleeper.app/v1/league/{league_id}/rosters").json()
users = requests.get(f"https://api.sleeper.app/v1/league/{league_id}/users").json()

roster_map = {r['roster_id']: r['owner_id'] for r in rosters}
user_map = {u['user_id']: u['display_name'] for u in users}

rookie_draft['owner_id'] = rookie_draft['roster_id'].map(roster_map)
rookie_draft['owner_name'] = rookie_draft['owner_id'].map(user_map)
del rookie_draft['owner_id']
del rookie_draft['roster_id']

rookie_draft.to_csv('rookie_draft.csv')

In [39]:
#finding the start date/time of the entry draft

league_id = "1125189236732284928"
drafts = requests.get(f"https://api.sleeper.app/v1/league/{league_id}/drafts").json()

startup_draft = drafts[0]  # assuming this is the entry draft
start_time_epoch = startup_draft['start_time']  # in UNIX epoch (milliseconds)

start_time = datetime.fromtimestamp(start_time_epoch / 1000)  # if in ms
print("Draft started at:", start_time)

Draft started at: 2024-08-11 18:01:07.164000


In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fetch the webpage content
url = "https://www.fantasypros.com/2024/08/superflex-dynasty-fantasy-football-draft-rankings-expert-picks-tiers/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Extract the table rows containing player data
rows = soup.find_all("tr")

# Initialize a list to hold the player data
data = []

# Iterate over each row and extract the relevant columns
for row in rows:
    cols = row.find_all("td")
    if len(cols) >= 5:  # Ensure the row has enough columns
        rank = cols[0].get_text(strip=True)
        tier = cols[1].get_text(strip=True)
        player_name = cols[2].get_text(strip=True)
        team = cols[3].get_text(strip=True)
        pos = cols[4].get_text(strip=True)
        bye_week = cols[5].get_text(strip=True)
        age = cols[6].get_text(strip=True)
        data.append([rank, tier, player_name, team, pos, bye_week, age])

# Create a DataFrame from the extracted data
df = pd.DataFrame(data, columns=["Rank", "Tier", "Player Name", "Team", "Position", "Bye Week", "Age"])

# Save the DataFrame to a CSV file
df.to_csv("superflex_dynasty_rankings_2024.csv", index=False)

print("CSV file has been created successfully.")


CSV file has been created successfully.


In [51]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fetch the webpage content
url = "https://www.fantasypros.com/nfl/rankings/dynasty-superflex.php"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Extract the table rows containing player data
rows = soup.find_all("tr")

# Initialize a list to hold the player data
data = []

# Iterate over each row and extract the relevant columns
for row in rows:
    cols = row.find_all("td")
    if len(cols) >= 5:  # Ensure the row has enough columns
        rank = cols[0].get_text(strip=True)
        tier = cols[1].get_text(strip=True)
        player_name = cols[2].get_text(strip=True)
        team = cols[3].get_text(strip=True)
        pos = cols[4].get_text(strip=True)
        bye_week = cols[5].get_text(strip=True)
        age = cols[6].get_text(strip=True)
        data.append([rank, tier, player_name, team, pos, bye_week, age])

# Create a DataFrame from the extracted data
df = pd.DataFrame(data, columns=["Rank", "Tier", "Player Name", "Team", "Position", "Bye Week", "Age"])

# Save the DataFrame to a CSV file
df.to_csv("superflex_dynasty_rankings.csv", index=False)

print("CSV file has been created successfully.")


CSV file has been created successfully.


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setup headless Chrome browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

url = "https://www.fantasypros.com/nfl/rankings/dynasty-superflex.php"
driver.get(url)

# Wait for page to fully load JS content (adjust if necessary)
time.sleep(25)

# Get the fully rendered HTML
html = driver.page_source
driver.quit()

soup = BeautifulSoup(html, "html.parser")

# Now find the table with rankings
table = soup.find("table")  # FantasyPros tables usually have id="data"

data = []
headers = [th.text.strip() for th in table.find("thead").find_all("th")]

for row in table.find("tbody").find_all("tr"):
    cols = [td.text.strip() for td in row.find_all("td")]
    data.append(cols)

df = pd.DataFrame(data, columns=headers)
df.to_csv("superflex_dynasty_rankings_selenium.csv", index=False)
print("CSV saved successfully!")


CSV saved successfully!
